In [1]:
%pip install gensim

Note: you may need to restart the kernel to use updated packages.


distutils: c:\Users\wulil\AppData\Local\Programs\Python\Python39\Include\UNKNOWN
sysconfig: c:\Users\wulil\AppData\Local\Programs\Python\Python39\Include
user = False
home = None
root = None
prefix = None
distutils: c:\Users\wulil\AppData\Local\Programs\Python\Python39\Include\UNKNOWN
sysconfig: c:\Users\wulil\AppData\Local\Programs\Python\Python39\Include
user = False
home = None
root = None
prefix = None
You should consider upgrading via the 'c:\Users\wulil\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
# Construct my gene2vec embedding layer
# how to implement embedding operation in scBERT
import gensim
import numpy as np
# import torch

# Tutorial

In [4]:
from gensim import corpora

documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

In [5]:
# remove common words and tokenize
stoplist = set('for a of the and to in'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in documents]

# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]

from pprint import pprint  # pretty-printer
pprint(texts)
# [['human', 'interface', 'computer'],
#  ['survey', 'user', 'computer', 'system', 'response', 'time'],
#  ['eps', 'user', 'interface', 'system'],
#  ['system', 'human', 'system', 'eps'],
#  ['user', 'response', 'time'],
#  ['trees'],
#  ['graph', 'trees'],
#  ['graph', 'minors', 'trees'],
#  ['graph', 'minors', 'survey']]

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [7]:
dictionary = corpora.Dictionary(texts)
dictionary.save('/deerwester.dict')  # store the dictionary, for future reference
print(dictionary)

# Dictionary(12 unique tokens)

Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...>


In [8]:
pprint(dictionary.token2id)

{'computer': 0,
 'eps': 8,
 'graph': 10,
 'human': 1,
 'interface': 2,
 'minors': 11,
 'response': 3,
 'survey': 4,
 'system': 5,
 'time': 6,
 'trees': 9,
 'user': 7}


## downsampling for ultrahigh freq genes
$$ P(w_i) = \max\left(1 - \sqrt{\frac{t}{f(w_i)}}, 0\right), $$

In [ ]:
def binning_expression(data, bins=200):
    """
    Binning the expression values into bins
    """
    data = np.log2(data + 1)
    data = np.digitize(data, np.linspace(0, 15, bins))
    return data

In [ ]:
# transformer
class Transformer(nn.Module):
    def __init__(self, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward=2048, dropout=0.1, activation="relu"):
        super(Transformer, self).__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward, dropout, activation)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_encoder_layers)
        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout, activation)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_decoder_layers)
        self.d_model = d_model
        self.nhead = nhead
        self.num_encoder_layers = num_encoder_layers
        self.num_decoder_layers = num_decoder_layers
        self.dim_feedforward = dim_feedforward
        self.dropout = dropout
        self.activation = activation
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        self.pos_decoder = PositionalEncoding(d_model, dropout)
        self.src_mask = None
        self.tgt_mask = None
        self.memory_mask = None
        self.src_key_padding_mask = None
        self.tgt_key_padding_mask = None
        self.memory_key_padding_mask = None

    def forward(self, src, tgt, src_mask=None, tgt_mask=None, memory_mask=None):
        src = self.pos_encoder(src)
        tgt = self.pos_decoder(tgt)
        output = self.transformer_encoder(src, src_mask, self.src_key_padding_mask)
        output = self.transformer_decoder(tgt, output, tgt_mask, memory_mask, self.tgt_key_padding_mask, self.memory_key_padding_mask)
        return output